In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interact, widgets
import matplotlib.dates as dates
from scipy.integrate import solve_ivp, solve_bvp
from IPython.display import Image
plt.style.use('seaborn-poster')
matplotlib.rcParams['figure.figsize'] = (10., 6.)
from scipy.special import lambertw, expit

In [ ]:
def x_inf(x,y,sigma):
    return -1./sigma * np.real(lambertw(-x*sigma*np.exp(-sigma*(x+y))))

def mu(x,y,sigma):
    return x*np.exp(-sigma*(x+y))

def dxinf_dy(x,y,sigma):
    xinf = x_inf(x,y,sigma)
    return -sigma*xinf/(1-sigma*xinf)

In [ ]:
sigma=3.
x = 0.9
y = 0.1
muu = mu(x,y,sigma)
xinf = x_inf(x,y,sigma)
print(dxinf_dy(x,y,sigma))
print(-sigma*muu/(np.exp(-sigma*xinf)-sigma*muu))

In [ ]:
# Tune these
c1 = 1.    # Cost of population that got infected
c2 = 1.0e-3   # Cost of control

beta = 0.3
gamma = 0.1
sigma0 = beta/gamma
qmax = 1.0

def rhs(t, u):
    # Variables: x, y, lambda_1, lambda_2
    du = np.zeros((4,len(t)))

    qstar = (u[3,:]-u[2,:])*beta*u[1,:]*u[0,:]/(2*c2)
    sigma = (1-qstar)*sigma0
    sigma = np.maximum((1-qmax)*sigma0,np.minimum(sigma0,sigma))

    du[0,:] = -sigma*gamma*u[1,:]*u[0,:]
    du[1,:] =  sigma*gamma*u[1,:]*u[0,:] - gamma*u[1]
    du[2,:] = (u[2,:]-u[3,:])*sigma*gamma*u[1,:]
    du[3,:] = (u[2,:]-u[3,:])*sigma*gamma*u[0,:] + u[3,:]*gamma
    return du

y0 = 0.01 # Initial infected
x0 = 0.99

def bc(ua, ub):
    xT = ub[0]; yT=ub[1]
    lam2T = -c1*dxinf_dy(xT,yT,sigma0)
    lam1T = lam2T*(1-1/(xT*sigma0))
    return np.array([ua[0]-x0, ua[1]-y0, ub[2]-lam1T, ub[3]-lam2T])

    
T = 100  # Final time
N = 1000
tt = np.linspace(0,T,N+1)
uu = np.zeros((4,N+1))
xT = 1./sigma0 + 0.05
yT = 0.
uselast=True
if uselast:
    last = result.sol(tt)
    uu[0,:] = last[0,:]
    uu[1,:] = last[1,:]
    uu[2,:] = last[2,:]
    uu[3,:] = last[3,:]
else:
    uu[0,:] = np.exp(-(beta-gamma)*tt/6)
    uu[1,:] = 0.5*np.exp(-1.e-3*(tt-15)**2)
    uu[2,:] = -c1*dxinf_dy(xT,yT,sigma0)*(1-1/(xT*sigma0))

result = solve_bvp(rhs, bc, tt, uu, max_nodes=2000000, tol=1.e-6, verbose=2)
x = result.y[0,:]
y = result.y[1,:]
lam1 = result.y[2,:]
lam2 = result.y[3,:]

qstar = (lam2-lam1)*beta*y*x/(2*c2)
sigma = (1-qstar)*sigma0
sigma = np.maximum((1-qmax)*sigma0,np.minimum(sigma0,sigma))
t = result.x
print(result.message)
obj = -c1*x_inf(x[-1],y[-1],sigma0) + c2*np.sum(np.diff(t)*(sigma0-sigma[1:])**2)
print(obj)
print(x_inf(x[-1],y[-1],sigma0))

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(t,x)
ax.plot(t,y)
ax.plot(t,1-sigma/sigma0)
ax.legend(['x','y','$\sigma/\sigma_0$']);
plt.xlabel('t')
plt.savefig('example1_time.pdf')

In [ ]:
N1 = 10; N2=5
Y, X = np.mgrid[0:1:100j, 0:1:100j]
U = -beta*X*Y
V = beta*X*Y - gamma*Y
x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma0,1,N2))
y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
seed_points = np.array([x_points, y_points])

plt.figure(figsize=(6,6))
plt.streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,
               broken_streamlines=False,linewidth=1)
plt.plot([0,1],[1,0],'-k',alpha=0.5)
plt.plot(x,y,'-k')
plt.plot([gamma/beta, gamma/beta],[0,1-gamma/beta],'--k',alpha=0.5)
plt.xlim(0,1); plt.ylim(0,1);
plt.xlabel('x'); plt.ylabel('y')
plt.savefig('example1_xy.pdf')

In [ ]:
plt.plot(t,lam1)
plt.plot(t,lam2)
plt.legend([r'$\lambda_1$',r'$\lambda_2$'])
plt.ylim(-0.05,0.05);

In [ ]:
# Tune these
c1 = 1.    # Cost of population that got infected

beta = 0.3
gamma = 0.1
sigma0 = beta/gamma

xs = []
ys = []
ts = []
qs = []
sigmas = []

c2s = np.array([1.,1.e-1,1.e-2,1.e-3,3.e-4,2.e-4,1.5e-4,1.e-5,1.e-6])
for c2 in c2s:

    def rhs(t, u):
        # Variables: x, y, lambda_1, lambda_2
        du = np.zeros((4,len(t)))

        sigma = sigma0 - (u[3,:]-u[2,:])*gamma*u[1,:]*u[0,:]/(2*c2)
        sigma = np.maximum(0,np.minimum(sigma0,sigma))

        du[0,:] = -sigma*gamma*u[1,:]*u[0,:]
        du[1,:] =  sigma*gamma*u[1,:]*u[0,:] - gamma*u[1]
        du[2,:] = (u[2,:]-u[3,:])*sigma*gamma*u[1,:]
        du[3,:] = (u[2,:]-u[3,:])*sigma*gamma*u[0,:] + u[3,:]*gamma
        return du

    y0 = 0.1 # Initial infected
    x0 = 0.9

    def bc(ua, ub):
        xT = ub[0]; yT=ub[1]
        lam2T = -c1*dxinf_dy(xT,yT,sigma0)
        lam1T = lam2T*(1-1/(xT*sigma0))
        return np.array([ua[0]-x0, ua[1]-y0, ub[2]-lam1T, ub[3]-lam2T])


    T = 100  # Final time
    N = 1000
    tt = np.linspace(0,T,N+1)
    uu = np.zeros((4,N+1))
    xT = 1./sigma0 + 0.05
    yT = 0.
    if c2<1:
        print(c2)
        last = result.sol(tt)
        uu[0,:] = last[0,:]
        uu[1,:] = last[1,:]
        uu[2,:] = last[2,:]
        uu[3,:] = last[3,:]
    else:
        uu[0,:] = np.exp(-(beta-gamma)*tt/6)
        uu[1,:] = 0.5*np.exp(-1.e-3*(tt-15)**2)
        uu[2,:] = c1

    result = solve_bvp(rhs, bc, tt, uu, max_nodes=2000000, tol=1.e-6, verbose=2)
    x = result.y[0,:]
    y = result.y[1,:]
    lam1 = result.y[2,:]
    lam2 = result.y[3,:]
    t = result.x
    
    sigma = sigma0 - (lam2-lam1)*gamma*y*x/(2*c2)
    sigma = np.maximum(0,np.minimum(sigma0,sigma))
    
    xs.append(x)
    ys.append(y)
    ts.append(t)
    sigmas.append(sigma)
    qs.append(1.-sigma/sigma0)

    print(result.message)

In [ ]:
fig, ax = plt.subplots(1,1)
palette = plt.get_cmap('tab10')
colors = ['red','brown','blue','purple','brown','green']
exps = ['-1','-2','-3','-5','-7','-8']
j=0
for i in [2,3,7]:#range(len(xs)):
    j += 1
    #ax.plot(ts[i],xs[i])
    ax.plot(ts[i],ys[i],color=palette(j),label='$c_2='+str(c2s[i])+'$')
    ax.plot(ts[i],sigmas[i]/sigma0,'--',color=palette(j))
#ax.legend(['x','y','$\sigma/\sigma_0$']);
plt.xlabel('t'); #plt.ylabel('$y(t)$ and $\sigma(t)/\sigma_0$')
plt.legend()
plt.savefig('varying_c2.pdf')

In [ ]:
N1 = 10; N2=5
Y, X = np.mgrid[0:1:100j, 0:1:100j]
U = -beta*X*Y
V = beta*X*Y - gamma*Y
x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma0,1,N2))
y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
seed_points = np.array([x_points, y_points])

plt.figure(figsize=(6,6))
plt.streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,
               broken_streamlines=False,linewidth=1)
plt.plot([0,1],[1,0],'-k',alpha=0.5)
j=0
for i in [2,3,7]:
    j+=1
    plt.plot(xs[i],ys[i],'-',color=palette(j))
plt.plot([gamma/beta, gamma/beta],[0,1-gamma/beta],'--k',alpha=0.5)
plt.xlim(0,1); plt.ylim(0,1);
plt.xlabel('x'); plt.ylabel('y')
plt.savefig('varying_c2_xy.pdf')

## Optimizing also for hospital capacity

In [ ]:
# Tune these
c1 = 1.    # Cost of population that got infected
c2 = 1.e-2   # Cost of control
c3 = 0.e-0

ymax = 0.1   # Hospital capacity
beta = 0.3
gamma = 0.1
sigma0 = beta/gamma


def rhs(t, u):
    # Variables: x, y, lambda_1, lambda_2
    du = np.zeros((4,len(t)))

    alpha = expit(10*(u[1,:]-ymax))*(u[1,:]-ymax)
        
    sigma = sigma0 - (u[3,:]-u[2,:])*gamma*u[1,:]*u[0,:]/(2*c2)
    sigma = np.maximum(0,np.minimum(sigma0,sigma))

    du[0,:] = -sigma*gamma*u[1,:]*u[0,:]
    du[1,:] =  sigma*gamma*u[1,:]*u[0,:] - gamma*u[1]
    du[2,:] = (u[2,:]-u[3,:])*sigma*gamma*u[1,:]
    du[3,:] = (u[2,:]-u[3,:])*sigma*gamma*u[0,:] + u[3,:]*gamma + c3*alpha
    return du

y0 = 0.01 # Initial infected
x0 = 0.99

def bc(ua, ub):
    xT = ub[0]; yT=ub[1]
    lam2T = -c1*dxinf_dy(xT,yT,sigma0)
    lam1T = lam2T*(1-1/(xT*sigma0))
    return np.array([ua[0]-x0, ua[1]-y0, ub[2]-lam1T, ub[3]-lam2T])

    
T = 100  # Final time
N = 1000
tt = np.linspace(0,T,N+1)
uu = np.zeros((4,N+1))
xT = 1./sigma0 + 0.05
yT = 0.
uselast=False
if uselast:
    last = result.sol(tt)
    uu[0,:] = last[0,:]
    uu[1,:] = last[1,:]
    uu[2,:] = last[2,:]
    uu[3,:] = last[3,:]
else:
    uu[0,:] = np.exp(-(beta-gamma)*tt/6)
    uu[1,:] = 0.5*np.exp(-1.e-3*(tt-15)**2)
    uu[2,:] = -c1#*dxinf_dy(xT,yT,sigma0)*(1-1/(xT*sigma0))
    #uu[3,:] = -c1*dxinf_dy(xT,yT,sigma0)

result = solve_bvp(rhs, bc, tt, uu, max_nodes=2000000, tol=1.e-6, verbose=2)
x = result.y[0,:]
y = result.y[1,:]
lam1 = result.y[2,:]
lam2 = result.y[3,:]

sigma = sigma0 - (lam2-lam1)*gamma*y*x/(2*c2)
sigma = np.maximum(0,np.minimum(sigma0,sigma))
t = result.x
print(result.message)
obj = -c1*x_inf(x[-1],y[-1],sigma0) + c2*np.sum(np.diff(t)*(sigma0-sigma[1:])**2)
print(obj)
print(x_inf(x[-1],y[-1],sigma0))

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(t,x)
ax.plot(t,y)
ax.plot(t,sigma/sigma0)
ax.legend(['x','y','$\sigma/\sigma_0$']);
plt.xlabel('t'); plt.ylabel('y(t) and q(t)');

In [ ]:
N1 = 10; N2=5
Y, X = np.mgrid[0:1:100j, 0:1:100j]
U = -beta*X*Y
V = beta*X*Y - gamma*Y
x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma0,1,N2))
y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
seed_points = np.array([x_points, y_points])

plt.figure(figsize=(6,6))
plt.streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,
               broken_streamlines=False,linewidth=1)
plt.plot([0,1],[1,0],'-k',alpha=0.5)
plt.plot(x,y,'-k')
plt.plot([gamma/beta, gamma/beta],[0,1-gamma/beta],'--k',alpha=0.5)
plt.xlim(0,1); plt.ylim(0,1);
plt.plot([0,1-ymax],[ymax,ymax],'--k',alpha=0.5)
plt.xlabel('x'); plt.ylabel('y');

In [ ]:
np.max(y)

## Function version

In [ ]:
def SIR_control(beta=0.3, gamma=0.1, x0=0.99, y0=0.01, c1=1., c2=1.e-2, c3=0., ymax=0.1, guess=None, T=100):
    sigma0 = beta/gamma

    def rhs(t, u):
        # Variables: x, y, lambda_1, lambda_2
        du = np.zeros((4,len(t)))

        alpha = expit(10*(u[1,:]-ymax))*(u[1,:]-ymax)
        
        sigma = sigma0 - (u[3,:]-u[2,:])*gamma*u[1,:]*u[0,:]/(2*c2)
        sigma = np.maximum(0,np.minimum(sigma0,sigma))

        du[0,:] = -sigma*gamma*u[1,:]*u[0,:]
        du[1,:] =  sigma*gamma*u[1,:]*u[0,:] - gamma*u[1]
        du[2,:] = (u[2,:]-u[3,:])*sigma*gamma*u[1,:]
        du[3,:] = (u[2,:]-u[3,:])*sigma*gamma*u[0,:] + u[3,:]*gamma - c3*alpha
        return du

    def bc(ua, ub):
        xT = ub[0]; yT=ub[1]
        lam2T = -c1*dxinf_dy(xT,yT,sigma0)
        lam1T = lam2T*(1-1/(xT*sigma0))
        return np.array([ua[0]-x0, ua[1]-y0, ub[2]-lam1T, ub[3]-lam2T])

    N = 1000
    tt = np.linspace(0,T,N+1)
    uu = np.zeros((4,N+1))
    xT = 1./sigma0 + 0.05
    yT = 0.
    if guess is not None:
        #last = result.sol(tt)
        uu[0,:] = guess[0,:]
        uu[1,:] = guess[1,:]
        uu[2,:] = guess[2,:]
        uu[3,:] = guess[3,:]
    else:
        uu[0,:] = np.exp(-(beta-gamma)*tt/6)
        uu[1,:] = 0.5*np.exp(-1.e-3*(tt-15)**2)
        uu[2,:] = -c1

    result = solve_bvp(rhs, bc, tt, uu, max_nodes=2000000, tol=1.e-6, verbose=2)
    x = result.y[0,:]
    y = result.y[1,:]
    lam1 = result.y[2,:]
    lam2 = result.y[3,:]

    sigma = sigma0 - (lam2-lam1)*gamma*y*x/(2*c2)
    sigma = np.maximum(0,np.minimum(sigma0,sigma))
    t = result.x
    print(result.message)
    print(obj)
    return x, y, sigma, t, result.sol(tt)

In [ ]:
x1, y1, sigma1, t1, newguess = SIR_control(c2=1.e-3,c3=10,T=100,guess=newguess)

In [ ]:
N1 = 10; N2=5
Y, X = np.mgrid[0:1:100j, 0:1:100j]
U = -beta*X*Y
V = beta*X*Y - gamma*Y
x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma0,1,N2))
y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
seed_points = np.array([x_points, y_points])

plt.figure(figsize=(6,6))
plt.streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,
               broken_streamlines=False,linewidth=1)
plt.plot([0,1],[1,0],'-k',alpha=0.5)
plt.plot(x1,y1,'-k')
plt.plot([gamma/beta, gamma/beta],[0,1-gamma/beta],'--k',alpha=0.5)
plt.xlim(0,1); plt.ylim(0,1);
plt.xlabel('x'); plt.ylabel('y');
#plt.legend()
plt.savefig('min_hosp_1_xy.pdf')

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(t1,x1)
ax.plot(t1,y1)
ax.plot(t1,sigma1/sigma0)
ax.legend(['x','y','$\sigma/\sigma_0$']);
plt.xlabel('t'); plt.ylabel('y(t) and q(t)');
plt.savefig('min_hosp_1_t.pdf')

In [ ]:
x1, y1, sigma1, t1, newguess = SIR_control(c2=1.e-3,c3=1,T=100,guess=newguess)

In [ ]:
N1 = 10; N2=5
Y, X = np.mgrid[0:1:100j, 0:1:100j]
U = -beta*X*Y
V = beta*X*Y - gamma*Y
x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma0,1,N2))
y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
seed_points = np.array([x_points, y_points])

plt.figure(figsize=(6,6))
plt.streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,
               broken_streamlines=False,linewidth=1)
plt.plot([0,1],[1,0],'-k',alpha=0.5)
plt.plot(x1,y1,'-k')
plt.plot([gamma/beta, gamma/beta],[0,1-gamma/beta],'--k',alpha=0.5)
plt.xlim(0,1); plt.ylim(0,1);
plt.xlabel('x'); plt.ylabel('y');
#plt.legend()
plt.savefig('min_hosp_2_xy.pdf')

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(t1,x1)
ax.plot(t1,y1)
ax.plot(t1,sigma1/sigma0)
ax.legend(['x','y','$\sigma/\sigma_0$']);
plt.xlabel('t'); plt.ylabel('y(t) and q(t)');
plt.savefig('min_hosp_2_t.pdf')

In [ ]:
x1, y1, sigma1, t1, newguess = SIR_control(c2=1.e-8,T=30)
x2, y2, sigma2, t2, newguess = SIR_control(c2=1.e-8,T=40)
x3, y3, sigma3, t3, newguess = SIR_control(c2=1.e-0,T=70)
x3, y3, sigma3, t3, newguess = SIR_control(c2=1.e-2,T=70,guess=newguess)
x3, y3, sigma3, t3, newguess = SIR_control(c2=1.e-3,T=70,guess=newguess)
x3, y3, sigma3, t3, newguess = SIR_control(c2=1.e-4,T=70,guess=newguess)
x3, y3, sigma3, t3, newguess = SIR_control(c2=1.e-8,T=70,guess=newguess)

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(t,x)
ax.plot(t,y)
ax.plot(t,sigma/sigma0)
ax.legend(['x','y','$\sigma/\sigma_0$']);
plt.xlabel('t'); plt.ylabel('y(t) and q(t)');

In [ ]:
N1 = 10; N2=5
Y, X = np.mgrid[0:1:100j, 0:1:100j]
U = -beta*X*Y
V = beta*X*Y - gamma*Y
x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma0,1,N2))
y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
seed_points = np.array([x_points, y_points])

plt.figure(figsize=(6,6))
plt.streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,
               broken_streamlines=False,linewidth=1)
plt.plot([0,1],[1,0],'-k',alpha=0.5)
plt.plot(x3,y3,'-',label='T='+str(Ts[2]))
plt.plot(x2,y2,'-',label='T='+str(Ts[1]))
plt.plot(x1,y1,'-',label='T='+str(Ts[0]))
plt.plot([gamma/beta, gamma/beta],[0,1-gamma/beta],'--k',alpha=0.5)
plt.xlim(0,1); plt.ylim(0,1);
plt.xlabel('x'); plt.ylabel('y');
plt.legend()
plt.savefig('diff-time-opt.pdf')

# Figure 7

In [ ]:
x1, y1, sigma1, t1, newguess = SIR_control(beta=0.3,gamma=0.1,c2=1.e-2,c3=0,T=100)

In [ ]:
N1 = 10; N2=5
Y, X = np.mgrid[0:1:100j, 0:1:100j]
U = -beta*X*Y
V = beta*X*Y - gamma*Y
x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma0,1,N2))
y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
seed_points = np.array([x_points, y_points])

plt.figure(figsize=(6,6))
plt.streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,
               broken_streamlines=False,linewidth=1)
plt.plot([0,1],[1,0],'-k',alpha=0.5)
#plt.plot(x3,y3,'-',label='T='+str(Ts[2]))
#plt.plot(x2,y2,'-',label='T='+str(Ts[1]))
plt.plot(x1,y1,'-',label='T='+str(Ts[0]))
plt.plot([gamma/beta, gamma/beta],[0,1-gamma/beta],'--k',alpha=0.5)
plt.xlim(0,1); plt.ylim(0,1);
plt.xlabel('x'); plt.ylabel('y');
plt.legend()
#plt.savefig('diff-time-opt.pdf')

# Real-world application for paper

In [ ]:
N = 1
alpha = 0.006  # IFR
eta = 2*alpha # Increase in IFR when no medical care is given
d = 1e4 # Days left of life for average victim
eps = 0.2  # Fraction of value of a day of life that is lost due to intervention
c1 = N*alpha
c2 = N*eps/d
c3 = eta*N
gamma = 1./10
sigma0 = 2.5
beta = sigma0*gamma
ymax=0.02
y0 = 1e-3
x0 = 1-y0
T = 200
x1, y1, sigma1, t1, newguess = SIR_control(beta=beta,gamma=gamma,c1=c1,c2=400*c2,c3=c3,T=T,guess=None,ymax=ymax)
x1, y1, sigma1, t1, newguess = SIR_control(beta=beta,gamma=gamma,c1=c1,c2=50*c2,c3=c3,T=T,guess=newguess,ymax=ymax)
x1, y1, sigma1, t1, newguess = SIR_control(beta=beta,gamma=gamma,c1=c1,c2=5*c2,c3=c3,T=T,guess=newguess,ymax=ymax)
x1, y1, sigma1, t1, newguess = SIR_control(beta=beta,gamma=gamma,c1=c1,c2=2.5*c2,c3=c3,T=T,guess=newguess,ymax=ymax)
x1, y1, sigma1, t1, newguess = SIR_control(beta=beta,gamma=gamma,c1=c1,c2=1.5*c2,c3=c3,T=T,guess=newguess,ymax=ymax)

x1, y1, sigma1, t1, newguess = SIR_control(beta=beta,gamma=gamma,c1=c1,c2=c2,c3=c3,T=T,
                                           guess=newguess,ymax=ymax,x0=x0,y0=y0)

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(t1,x1)
ax.plot(t1,y1)
ax.plot(t1,sigma1/sigma0)
ax.legend(['x','y','$\sigma/\sigma_0$']);
plt.xlabel('t');
ax.autoscale(enable=True, axis='x', tight=True)
plt.savefig('real_world_1_t.pdf')

In [ ]:
N1 = 10; N2=5
Y, X = np.mgrid[0:1:100j, 0:1:100j]
U = -beta*X*Y
V = beta*X*Y - gamma*Y
x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma0,1,N2))
y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
seed_points = np.array([x_points, y_points])

plt.figure(figsize=(6,6))
plt.streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,
               broken_streamlines=False,linewidth=1)
plt.plot([0,1],[1,0],'-k',alpha=0.5)
plt.plot(x1,y1,'-k')
plt.plot([gamma/beta, gamma/beta],[0,1-gamma/beta],'--k',alpha=0.5)
plt.xlim(0,1); plt.ylim(0,1);
plt.xlabel('x'); plt.ylabel('y');
#plt.legend()
plt.savefig('real_world_1_xy.pdf')

In [ ]:
np.max(y1)

In [ ]:
print(c1,c2,c3)

In [ ]:
N = 1
alpha = 0.012  # IFR
eta = 2*alpha # Increase in IFR when no medical care is given
d = 1e4 # Days left of life for average victim
eps = 0.05 # Fraction of value of a day of life that is lost due to intervention
c1 = N*alpha
c2 = N*eps/d
c3 = eta*N
y0 = 1e-3
x0 = 1-y0
T = 200

x1, y1, sigma1, t1, newguess = SIR_control(beta=beta, gamma=gamma,c1=c1,c2=100*c2,c3=c3,T=T,guess=None,x0=x0,y0=y0)
x1, y1, sigma1, t1, newguess = SIR_control(beta=beta, gamma=gamma,c1=c1,c2=10*c2,c3=c3,T=T,guess=newguess,x0=x0,y0=y0)
x1, y1, sigma1, t1, newguess = SIR_control(beta=beta, gamma=gamma,c1=c1,c2=3*c2,c3=c3,T=T,guess=newguess,x0=x0,y0=y0)
x1, y1, sigma1, t1, newguess = SIR_control(beta=beta, gamma=gamma,c1=c1,c2=c2,c3=c3,T=T,guess=newguess,x0=x0,y0=y0)

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(t1,x1)
ax.plot(t1,y1)
ax.plot(t1,sigma1/sigma0)
ax.legend(['x','y','$\sigma/\sigma_0$']);
plt.xlabel('t');
plt.savefig('real_world_2_t.pdf')

In [ ]:
N1 = 10; N2=5
Y, X = np.mgrid[0:1:100j, 0:1:100j]
U = -beta*X*Y
V = beta*X*Y - gamma*Y
x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma0,1,N2))
y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
seed_points = np.array([x_points, y_points])

plt.figure(figsize=(6,6))
plt.streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,
               broken_streamlines=False,linewidth=1)
plt.plot([0,1],[1,0],'-k',alpha=0.5)
plt.plot(x1,y1,'-k')
plt.plot([gamma/beta, gamma/beta],[0,1-gamma/beta],'--k',alpha=0.5)
plt.xlim(0,1); plt.ylim(0,1);
plt.xlabel('x'); plt.ylabel('y');
#plt.legend()
plt.savefig('real_world_2_xy.pdf')

In [ ]:
np.max(y1)

In [ ]:
print(c1,c2,c3)

In [ ]:
N = 1
alpha = 0.006  # IFR
eta = 2*0.006 # Increase in IFR when no medical care is given
d = 1e4 # Days left of life for average victim
eps = 0.5 # Fraction of value of a day of life that is lost due to intervention
c1 = N*alpha
c2 = N*eps/d
c3 = eta*N
T=200
x1, y1, sigma1, t1, newguess = SIR_control(beta=beta, gamma=gamma,c1=c1,c2=20*c2,c3=c3,T=T,guess=None)

x1, y1, sigma1, t1, newguess = SIR_control(beta=beta, gamma=gamma,c1=c1,c2=3*c2,c3=c3,T=T,guess=newguess)
x1, y1, sigma1, t1, newguess = SIR_control(beta=beta, gamma=gamma,c1=c1,c2=c2,c3=c3,T=T,guess=newguess,x0=x0,y0=y0)

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(t1,x1)
ax.plot(t1,y1)
ax.plot(t1,sigma1/sigma0)
ax.legend(['x','y','$\sigma/\sigma_0$']);
plt.xlabel('t');
plt.savefig('real_world_3_t.pdf')

In [ ]:
N1 = 10; N2=5
Y, X = np.mgrid[0:1:100j, 0:1:100j]
U = -beta*X*Y
V = beta*X*Y - gamma*Y
x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma0,1,N2))
y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
seed_points = np.array([x_points, y_points])

plt.figure(figsize=(6,6))
plt.streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,
               broken_streamlines=False,linewidth=1)
plt.plot([0,1],[1,0],'-k',alpha=0.5)
plt.plot(x1,y1,'-k')
plt.plot([gamma/beta, gamma/beta],[0,1-gamma/beta],'--k',alpha=0.5)
plt.xlim(0,1); plt.ylim(0,1);
plt.xlabel('x'); plt.ylabel('y');
#plt.legend()
plt.savefig('real_world_3_xy.pdf')

In [ ]:
print(c1,c2,c3)

In [ ]:
beta

# Second wave control

In [ ]:
N = 1
alpha = 0.02  # IFR
eta = 0.02 # Increase in IFR when no medical care is given
d = 1e4 # Days left of life for average victim
eps = 0.1 # Fraction of value of a day of life that is lost due to intervention
c1 = N*alpha
c2 = N*eps/d
c3 = eta*gamma*N
x1, y1, sigma1, t1, newguess = SIR_control(x0=0.8,c1=c1,c2=20*c2,c3=c3,T=100,guess=None)

x1, y1, sigma1, t1, newguess = SIR_control(x0=0.8,c1=c1,c2=3*c2,c3=c3,T=100,guess=newguess)
x1, y1, sigma1, t1, newguess = SIR_control(x0=0.8,c1=c1,c2=c2,c3=c3,T=100,guess=newguess)

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(t1,x1)
ax.plot(t1,y1)
ax.plot(t1,sigma1/sigma0)
ax.legend(['x','y','$\sigma/\sigma_0$']);
plt.xlabel('t');
plt.savefig('second_wave_t.pdf')

In [ ]:
N1 = 10; N2=5
Y, X = np.mgrid[0:1:100j, 0:1:100j]
U = -beta*X*Y
V = beta*X*Y - gamma*Y
x_points = list(np.linspace(0,1,N1)) + list(np.linspace(1./sigma0,1,N2))
y_points = list(1.-np.linspace(0,1,N1)) + [1.e-6]*N2
seed_points = np.array([x_points, y_points])

plt.figure(figsize=(6,6))
plt.streamplot(X, Y, U, V, start_points=seed_points.T,integration_direction='forward',maxlength=1000,
               broken_streamlines=False,linewidth=1)
plt.plot([0,1],[1,0],'-k',alpha=0.5)
plt.plot(x1,y1,'-k')
plt.plot([gamma/beta, gamma/beta],[0,1-gamma/beta],'--k',alpha=0.5)
plt.xlim(0,1); plt.ylim(0,1);
plt.xlabel('x'); plt.ylabel('y');
#plt.legend()
plt.savefig('real_world_3.pdf')